In [ ]:
# cp xyGrab.als xyGrab.gz; gzip -d xyGrab.gz -f

In [ ]:
import xml.etree.ElementTree as ET
import gzip
def getAutomationsByTrackDeviceClipParam(trackNum, deviceName, clipName, paramName, tree):
    tracks = []
    for e in tree.iter():
        if e.tag == "MidiTrack":
            tracks.append(e)
            
    devices = []
    for e in tracks[trackNum].iter():
        if e.tag == "MxDeviceAudioEffect":
            name = e.find("UserName").attrib["Value"]
            if name == deviceName:
                devices.append(e)
                
    device = devices[0]
    param_name_to_id = {}
    for e in device.iter():
        if e.tag == "MxDFloatParameter":
            name = e.find("Name").attrib["Value"]
            param_id = e.attrib["Id"]
            automation_target_id = e.find("Timeable/AutomationTarget").attrib["Id"]
            param_name_to_id[name] = automation_target_id
    
    param_id_to_curve = {}
    for e in tree2.iter():
        if e.tag == "ClipEnvelope":
            pointee_id = e.find("EnvelopeTarget/PointeeId").attrib["Value"]
            param_id_to_curve[pointee_id] = e.find("Automation/Events")
    
    return param_id_to_curve[param_name_to_id[paramName]]

In [ ]:
tree2 = ET.parse(gzip.open('xyGrab Project/xyGrab.als'))
getAutomationsByTrackDeviceClipParam(0, "map_target", "1-Operator 6", "p-1", tree2)

In [ ]:
# findall LiveSet/Tracks/MidiTrack
# in each MidiTrack dig into DeviceChain/MainSequencer/ClipSlotList
# in the clip slot list findall ClipSlot/ClipSlot/Value/MidiClip/
# names are in MidiClip/Name and envelopes are in MidiClip/Envelopes/Envelopes/ClipEnvelope
# this will let you get the names of all MidiClips (there are ones in TakeLanes subtrees we want to ignore)

# alternatively, just go through whole XML doc for MidiClips and ignore those without automations
# hidden take-lane clips probs won't have automations (and probs won't have duplicate names)

#json object will be clipName: {x: p-0-data, y: p-1-data}

# get all the MxDFloatParameter id numbers by just full traversal thru whole XML doc

In [ ]:
import json

def xml2jsonCurve(xml_curve):
    return

def assembleCurve(x_curve, y_curve):
    x_clean = xml2jsonCurve(x_curve)
    y_clean = xml2jsonCurve(y_curve)
    return
        

def getCurvesJson(tree):
    clip_curves = {}
    param_id_to_name = {}
    for e in tree.iter():
        if e.tag == "MxDFloatParameter":
            name = e.find("Name").attrib["Value"]
            automation_target_id = e.find("Timeable/AutomationTarget").attrib["Id"]
            param_id_to_name[automation_target_id] = name
    
    for e in tree.iter():
        if e.tag == "MidiClip":
            clip_name = e.find("Name").attrib["Id"]
            clip_curves[clip_name] = {}
            x_curve = []
            y_curve = []
            for e2 in e.iter():
                if e2.tag == "ClipEnvelope":
                    automation_target_id = e.find("EnvelopeTarget/PointeeId").attrib["Value"]
                    param_name = param_id_to_name[automation_target_id]
                    curve_xml = e.find("Automation/Events")
                    if param_name = "p-0":
                        x_curve = curve_xml
                    if param_name = "p-1":
                        y_curve = curve_xml
            if len(x_curve) > 0 and len(y_curve) > 0:
                clipCurves[clip_name] = assembleCurve(x_curve, y_curve)
        
    return json.dumps(clip_curves)
            
                    
    
            
    

In [ ]:
server.shutdown()
server.server_close()

In [ ]:
# https://docs.python.org/3/library/socketserver.html#socketserver-tcpserver-example
import socketserver

class MyTCPHandler(socketserver.StreamRequestHandler):

    def handle(self):
        # self.rfile is a file-like object created by the handler;
        # we can now use e.g. readline() instead of raw recv() calls
        self.data = self.rfile.readline().strip()
        print("{} wrote:".format(self.client_address[0]))
        print(self.data)
        # Likewise, self.wfile is a file-like object used to write back
        # to the client
        self.wfile.write(self.data + bytes(b"\n"))


HOST, PORT = "localhost", 11999

# Create the server, binding to localhost on port 9999
with socketserver.TCPServer((HOST, PORT), MyTCPHandler) as server:
    # Activate the server; this will keep running until you
    # interrupt the program with Ctrl-C
    server.serve_forever()